In [1]:
import pandas as pd
import numpy as np
import re
import time
import json
import random
import gensim
import jieba

E:\Applications\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
typelist=['犯罪', '喜剧', '舞台艺术', '新闻', '儿童', '真人秀', '歌舞', '戏曲', '音乐', '冒险', '灾难', '恐怖', '家庭', '短片', '历史', '运动', '动画', '纪录片', '惊悚', '科幻', '战争', '动作', '健康', '脱口秀', '悬疑', '剧情', '古装', '鬼怪', '奇幻', '武侠', '同性', '体育', '西部', '传记', '爱情']
typelist=sorted(typelist)
print(typelist)

['传记', '体育', '健康', '儿童', '冒险', '剧情', '动作', '动画', '历史', '古装', '同性', '喜剧', '奇幻', '家庭', '恐怖', '悬疑', '惊悚', '戏曲', '战争', '新闻', '歌舞', '武侠', '灾难', '爱情', '犯罪', '真人秀', '短片', '科幻', '纪录片', '脱口秀', '舞台艺术', '西部', '运动', '音乐', '鬼怪']


In [4]:
dy={'传记':'成年',
 '体育':'男性',
 '健康':'老人',
 '儿童':'儿童',
 '冒险':'男性',
 '剧情':'成年',
 '动作':'男性',
 '动画':'儿童',
 '历史':'成年',
 '古装':'成年',
 '同性':'女性',
 '喜剧':'无',
 '奇幻':'无',
 '家庭':'女性',
 '恐怖':'成年',
 '悬疑':'男性',
 '惊悚':'男性',
 '戏曲':'老人',
 '战争':'男性',
 '新闻':'成年',
 '歌舞':'女性',
 '武侠':'男性',
 '灾难':'男性',
 '爱情':'女性',
 '犯罪':'男性',
 '真人秀':'成年',
 '短片':'女性',
 '科幻':'男性',
 '纪录片':'男性',
 '脱口秀':'成年',
 '舞台艺术':'成年',
 '西部':'男性',
 '运动':'男性',
 '音乐':'成年',
 '鬼怪':'男性'}

In [10]:
wu=pd.read_csv('test/wvusermatrix.csv',encoding='gbk')#word2vec用户矩阵
wt=pd.read_csv('wvtvmatrix.csv',encoding='gbk')#word2vec节目矩阵
tu=pd.read_csv('test/tagusermatrix.csv',encoding='gbk')#标签用户矩阵
cu=pd.read_csv('channelusermatrix.csv',encoding='gbk')#标签用户频道矩阵
tt=pd.read_csv('tagtvmatrix.csv',encoding='gbk')#标签节目矩阵
dfr=pd.read_csv('test/mingzilianhe.csv',encoding='gbk')#用户看过的节目
info=pd.read_csv('info2.csv',encoding='gbk').fillna('')#节目信息
for i in range(len(tu)):
    if np.sum(tu.loc[i,:])!=0:
        tu.loc[i,:]=tu.loc[i,:]/np.sum(tu.loc[i,:])
for i in range(len(cu)):
    if np.sum(cu.loc[i,:])!=0:
        cu.loc[i,:]=cu.loc[i,:]/np.sum(cu.loc[i,:])
tu=tu+cu*0.5

In [12]:
utag=pd.DataFrame(np.zeros((1329,6)),columns=['老人','成年','男性','女性','儿童','无'])
for i in range(1329):
    for t in typelist:
        utag.loc[i,dy[t]]+=tu.loc[i,t]

In [14]:
for i in range(len(utag)):
    utag.loc[i,:]/=np.sum(utag.loc[i,:])

In [15]:
utag

,老人,成年,男性,女性,儿童,无
0,0.000000,0.400878,0.261739,0.180836,0.000000,0.156548
1,0.000000,0.476084,0.141712,0.098174,0.279052,0.004978
2,0.022903,0.480308,0.170458,0.262532,0.000000,0.063799
3,0.005183,0.859843,0.065169,0.054350,0.009929,0.005526
4,0.006530,0.741681,0.141080,0.081915,0.014253,0.014542
5,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN
7,0.000000,0.537756,0.338674,0.022870,0.097753,0.002947
8,0.000000,0.234653,0.530694,0.234653,0.000000,0.000000
9,0.006926,0.242207,0.043327,0.479472,0.003071,0.224997


In [20]:
from collections import Counter

words = [
    'look', 'into', 'my', 'eyes', 'look', 'into', 'my', 'eyes',
    'the',  'the', 'the',  'not', 'around', 'the',
    'eyes', "don't", 'look', 'around', 'the', 'eyes', 'look', 'into',
    'my', 'eyes', "you're", 'under'
]

print (Counter(words).most_common())

[('eyes', 5), ('the', 5), ('look', 4), ('my', 3), ('into', 3), ('around', 2), ('not', 1), ('under', 1), ("don't", 1), ("you're", 1)]


In [25]:
ulist=['老人','成年','男性','女性','儿童','无']
usertag=pd.DataFrame(columns=['用户','tag1','tag2','tag3','tag4'])
n=0
for i in range(len(utag)):
    for u in ulist[:-1]:
        if utag.loc[i,u]>0.166:
            usertag.loc[n,'用户']=i+10001
            usertag.loc[n,'tag1']='基本特征'
            usertag.loc[n,'tag2']='家庭成员'
            if u=='男性' or u=='女性':
                usertag.loc[n,'tag3']='成年'
                usertag.loc[n,'tag4']=u
            else:
                usertag.loc[n,'tag3']=u
                usertag.loc[n,'tag4']=''
            n+=1

In [26]:
usertag

,用户,tag1,tag2,tag3,tag4
0,10001,基本特征,家庭成员,成年,
1,10001,基本特征,家庭成员,成年,男性
2,10001,基本特征,家庭成员,成年,女性
3,10002,基本特征,家庭成员,成年,
4,10002,基本特征,家庭成员,儿童,
5,10003,基本特征,家庭成员,成年,
6,10003,基本特征,家庭成员,成年,男性
7,10003,基本特征,家庭成员,成年,女性
8,10004,基本特征,家庭成员,成年,
9,10005,基本特征,家庭成员,成年,


In [28]:
usertag.to_csv('result/家庭成员.csv',index=False)